# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
list(data)

['id',
 'ad',
 'education',
 'ofjobs',
 'yearsexp',
 'honors',
 'volunteer',
 'military',
 'empholes',
 'occupspecific',
 'occupbroad',
 'workinschool',
 'email',
 'computerskills',
 'specialskills',
 'firstname',
 'sex',
 'race',
 'h',
 'l',
 'call',
 'city',
 'kind',
 'adid',
 'fracblack',
 'fracwhite',
 'lmedhhinc',
 'fracdropout',
 'fraccolp',
 'linc',
 'col',
 'expminreq',
 'schoolreq',
 'eoe',
 'parent_sales',
 'parent_emp',
 'branch_sales',
 'branch_emp',
 'fed',
 'fracblack_empzip',
 'fracwhite_empzip',
 'lmedhhinc_empzip',
 'fracdropout_empzip',
 'fraccolp_empzip',
 'linc_empzip',
 'manager',
 'supervisor',
 'secretary',
 'offsupport',
 'salesrep',
 'retailsales',
 'req',
 'expreq',
 'comreq',
 'educreq',
 'compreq',
 'orgreq',
 'manuf',
 'transcom',
 'bankreal',
 'trade',
 'busservice',
 'othservice',
 'missind',
 'ownership']

In [6]:
data_all = data[['race', 'call']]
data_all


,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0
5,w,0.0
6,w,0.0
7,b,0.0
8,b,0.0
9,b,0.0


In [7]:
#total black applicants
b_app = data_all[data_all.race == 'b'].race.size
b_app

2435

In [8]:
#total white applicants
w_app = data_all[data_all.race == 'w'].race.size
w_app

2435

In [9]:
#black applicants that were called back
b_called=sum(data_all[data_all.race=='b'].call)
b_called

157.0

In [10]:
#white applicants that were called back
w_called=sum(data_all[data_all.race=='w'].call)
w_called

235.0

In [11]:
#percentages
per_w_called = (w_called/ w_app)*100
per_b_called = (b_called/ b_app)*100
diff = per_w_called - per_b_called
print(round(per_w_called,2), "% of white applicants were called back.\n",
      round(per_b_called,2), "% of black applicants were called back.\n",
      "This is a difference of ", round(diff,2), "%.")
      

9.65 % of white applicants were called back.
 6.45 % of black applicants were called back.
 This is a difference of  3.2 %.


### Question 1
What test is appropriate for this problem? Does CLT apply?

<p>Use a z-test because we have more that 30 observations in our data set. CLT does apply in this case because the percentage called back will follow a normal distribution.</p>

### Question 2
What are the null and alternate hypotheses?

<p> Answer: </p>
<p>Ho: There is no difference between black and white resumes/ There is no significant difference between "percentage called back" for black and white resumes.</p>
<p>H1: There is difference between black and white resumes/ There is significant difference between "percentage called back" for black and white resumes.</p>

### Question 3
Compute margin of error, confidence interval, and p-value.

In [23]:
#http://www.dummies.com/education/math/statistics/how-to-calculate-the-margin-of-error-for-a-sample-proportion/
#For 95% CI, z-value = 1.96

P1=per_b_called/100
n1=b_app
var_b=(P1*(1-P1)/n1)
var_b

P2=per_w_called/100
n2=w_app
var_w=(P2*(1-P2)/n2)
var_w


3.5809119833046381e-05

In [24]:
#variance
var_bw= var_b + var_w
var_bw

6.0580857689544847e-05

In [25]:
#standard error
std_err=np.sqrt(var_bw)
std_err


0.0077833705866767544

In [26]:
#margin of error
moe = std_err*1.96
moe
print("The margin of error is", round(moe,2))

The margin of error is 0.02


In [27]:
## confidence interval
ci = abs(P1-P2) + np.array([-1, 1]) * moe
ci

array([ 0.01677745,  0.04728826])

In [28]:
## degrees of freedom
b1=var_b/n1
w1=var_w/n2

df=((b1+w1)**2)/(((b1**2)/n1)+((w1**2)/n2))
df

4713.5381934322604

In [29]:
#t-value calculation
t_val=((P1-P2)-0)/std_err
t_val

-4.1155504357300003

In [34]:
p_val = stats.t.sf(np.abs(t_val), df)*2  
p_val
print("The p-value is", round(p_val, 7))

The p-value is 3.93e-05


### Question 4
Write a story describing the statistical significance in the context or the original problem.

In the professional world, there is supposed inequality regarding the rate of call backs received by respective races. Specifically, the difference between the callback rate of white vs. black applicants seems to be fairly large. In this exercise, a z-test was used to determine if the difference in callback rates was statistically significant. After computing the p-value, because it was a very low value, the null hypothesis can be rejected. This means that H1 is accepted and "There is difference between black and white resumes/ There is significant difference between "percentage called back" for black vs. white resumes."

### Question 5
Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Further analysis would be necessary to see if this was the most important factor relative to callback success. This test only says that race contributes to callback success, but not by how much. It is possible that multiple factors could be identified as possibly affecting callback success. For each factor, a z-test could be performed and determined if it is statistcally significant.